In [2]:
import os
import boto3
import numpy as np

s3_client = boto3.client('s3')

def download_s3_file(bucket, folder, filename, s3_client):
    if not os.path.exists('/tmp/' + folder):
        os.mkdir('/tmp/' + folder)

    key = folder + "/" + filename # e.g. 'S3_U0_m2'
    s3_client.download_file(bucket, key, '/tmp/' + key)
    return '/tmp/' + key


def get_all_objects_for_matrix(matrix_name):
    return s3_client.list_objects_v2(
        Bucket='jmue-multiplication-benchmarks',
        Delimiter='string',
        Prefix=matrix_name + '/'
    )

In [27]:
response = get_all_objects_for_matrix('sq_8kx8k_bs4k')

In [31]:
print response['KeyCount']
response

4


{u'Contents': [{u'ETag': '"4cfb819a40e4862fbcd7b9f477177dd8-16"',
   u'Key': 'sq_8kx8k_bs4k/m_0_0.npy',
   u'LastModified': datetime.datetime(2017, 10, 14, 12, 57, 41, tzinfo=tzutc()),
   u'Size': 128000080,
   u'StorageClass': 'STANDARD'},
  {u'ETag': '"3598a8e80ac99125d1fb7bff9403b731-16"',
   u'Key': 'sq_8kx8k_bs4k/m_0_1.npy',
   u'LastModified': datetime.datetime(2017, 10, 14, 12, 57, 49, tzinfo=tzutc()),
   u'Size': 128000080,
   u'StorageClass': 'STANDARD'},
  {u'ETag': '"7c666644cb2ce845a86963df26756305-16"',
   u'Key': 'sq_8kx8k_bs4k/m_1_0.npy',
   u'LastModified': datetime.datetime(2017, 10, 14, 12, 57, 56, tzinfo=tzutc()),
   u'Size': 128000080,
   u'StorageClass': 'STANDARD'},
  {u'ETag': '"44cafe001fe513cf0f1be63408bc0fbf-16"',
   u'Key': 'sq_8kx8k_bs4k/m_1_1.npy',
   u'LastModified': datetime.datetime(2017, 10, 14, 12, 58, 4, tzinfo=tzutc()),
   u'Size': 128000080,
   u'StorageClass': 'STANDARD'}],
 u'Delimiter': 'string',
 u'IsTruncated': False,
 u'KeyCount': 4,
 u'MaxKey

In [33]:
np.hstack(([[1,2,3],[1,1,1]], [[6,6,6],[6,6,6]]))

array([[1, 2, 3, 6, 6, 6],
       [1, 1, 1, 6, 6, 6]])

In [35]:
m_0_0 = np.load('/Volumes/data/sq_4kx4k_bs2k/m_0_0.npy')
m_0_1 = np.load('/Volumes/data/sq_4kx4k_bs2k/m_0_1.npy')
m_1_0 = np.load('/Volumes/data/sq_4kx4k_bs2k/m_1_0.npy')
m_1_1 = np.load('/Volumes/data/sq_4kx4k_bs2k/m_1_1.npy')

In [3]:
m_0_0 = np.array([[1,1,1,1],[1,1,1,1],[1,1,1,1],[1,1,1,1]])
m_0_1 = np.array([[2,2,2,2],[2,2,2,2],[2,2,2,2],[2,2,2,2]])
m_1_0 = np.array([[3,3,3,3],[3,3,3,3],[3,3,3,3],[3,3,3,3]])
m_1_1 = np.array([[4,4,4,4],[4,4,4,4],[4,4,4,4],[4,4,4,4]])

In [4]:
top = np.hstack([m_0_0, m_0_1])
bottom = np.hstack((m_1_0, m_1_1))
complete = np.vstack((top, bottom))

In [5]:
top

array([[1, 1, 1, 1, 2, 2, 2, 2],
       [1, 1, 1, 1, 2, 2, 2, 2],
       [1, 1, 1, 1, 2, 2, 2, 2],
       [1, 1, 1, 1, 2, 2, 2, 2]])

In [43]:
complete

array([[1, 1, 1, 1, 2, 2, 2, 2],
       [1, 1, 1, 1, 2, 2, 2, 2],
       [1, 1, 1, 1, 2, 2, 2, 2],
       [1, 1, 1, 1, 2, 2, 2, 2],
       [3, 3, 3, 3, 4, 4, 4, 4],
       [3, 3, 3, 3, 4, 4, 4, 4],
       [3, 3, 3, 3, 4, 4, 4, 4],
       [3, 3, 3, 3, 4, 4, 4, 4]])

In [56]:
top, bottom = np.vsplit(complete, 2)

In [57]:
top

array([[1, 1, 1, 1, 2, 2, 2, 2],
       [1, 1, 1, 1, 2, 2, 2, 2],
       [1, 1, 1, 1, 2, 2, 2, 2],
       [1, 1, 1, 1, 2, 2, 2, 2]])

In [58]:
bottom

array([[3, 3, 3, 3, 4, 4, 4, 4],
       [3, 3, 3, 3, 4, 4, 4, 4],
       [3, 3, 3, 3, 4, 4, 4, 4],
       [3, 3, 3, 3, 4, 4, 4, 4]])

In [59]:
m_0_0, m_0_1 = np.hsplit(top, 2)
m_1_0, m_1_1 = np.hsplit(bottom, 2)

In [34]:
def build_matrix(matrix_name, dimension, block_size):
    rows = []
    for i in range(0,dimension/block_size):
        blocks = []
        for j in range(0,dimension/block_size):
            blocks.append(np.load('/tmp/{}/m_{}_{}.npy'.format(matrix_name,i,j)))
        rows.append(np.vstack(blocks))
    return np.hstack(rows)

In [38]:
matrix = build_matrix('sq_4kx4k_bs2k', 4, 2)

In [39]:
matrix.shape

(4000, 4000)

In [40]:
np.array([[1,2,3,4],[1,2,3,4],[1,2,3,4],[1,2,3,4]]).shape

(4, 4)

In [44]:
for i, val in enumerate([5,5,5,5]):
    print val
    print i

5
0
5
1
5
2
5
3
